In [1]:
import torch 
import torch.nn as nn
import fasttext as ft
import math
import bcolz
import numpy as np
import pickle

# English Word Embedding Preprocessing

In [ ]:
# Do not run
# words = []
# idx = 0
# word2idx = {}
# vectors = bcolz.carray(np.zeros(1), rootdir=f'english_6B.300.dat', mode='w')

In [ ]:
# Do not run
# with open(f'english_glove.6B.300d.txt', 'rb') as f:
#     for l in f:
#         line = l.decode().split()
#         word = line[0]
#         words.append(word)
#         word2idx[word] = idx
#         idx += 1
#         vect = np.array(line[1:]).astype(np.float)
#         vectors.append(vect)

In [ ]:
# Do not run
# vectors = bcolz.carray(vectors[1:].reshape((-1, 300)), rootdir=f'english_6B.300.dat', mode='w')
# vectors.flush()
# pickle.dump(words, open(f'english_words.pkl', 'wb'))
# pickle.dump(word2idx, open(f'english_word2idx.pkl', 'wb'))

In [ ]:
# english_vectors = bcolz.open(f'english_6B.300.dat')[:]
english_words = pickle.load(open(f'english_words.pkl', 'rb'))
# english_word2idx = pickle.load(open(f'english_word2idx.pkl', 'rb'))
# english_vectors = pickle.load(open(f'english_vectors.pkl', 'rb'))
# len(english_vectors), len(english_words), len(english_word2idx)
# english_glove = {w: english_vectors[english_word2idx[w]] for w in english_words}

In [ ]:
# To find the index of a word in vocabulary
# english_word2idx['where']
# english_glove['the']

In [ ]:
# One Hot encoding
# onehotdict = {}
# for a,b in enumerate(english_words):
#     onehotdict[b] = torch.zeros(1,400001).float()


In [ ]:
# pickle.dump(english_vectors, open(f'english_vectors.pkl', 'wb'))
# english_vectors.shape

In [ ]:
english_word2idx['automatic']

In [ ]:
english_words[20:50]

Subsampling

In [ ]:
indices = []
ind = 0
count = 1
englishSens = []
with open('train.en','r') as f:
    for line in f:
        flag = 'green'
        linelist = line.split()
        for word in linelist:
            if word.lower() not in english_words:
#                 print(word)
                flag = 'red'
        if flag == 'green':
            englishSens.append(line.split('\n')[0])
            indices.append(ind)
            count+=1
        ind+=1
        if count == 101:
            break

In [5]:
for i in englishSens:
    l = i.split()
    for j in l:
        if j.lower() not in english_words:
            print(j)

NameError: name 'englishSens' is not defined

In [4]:
gindices = []
gind = 0
count = 1
germanSens = []
with open('train.de','r') as f:
    for line in f:
        flag = 'green'
        linelist = line.split()
        for word in linelist:
            if word.lower() not in german_words:
#                 print(word)
                flag = 'red'
        if flag == 'green':
            germanSens.append(line.split('\n')[0])
            gindices.append(gind)
            count+=1
        gind+=1
        if count == 101:
            break

TypeError: '_io.TextIOWrapper' object is not subscriptable

In [ ]:
germanSens = []
gind = 0
with open('train.de','r') as f:
    for line in f:
        if gind in indices:
            germanSens.append(line.split('\n')[0])
        gind+=1
        if gind > max(indices):
            break

In [ ]:
ef = open('train.de','r')
er = ef.read().split('\n')

In [ ]:
max([1

# German Word Embedding Preprocessing

In [ ]:
# words = []
# idx = 0
# word2idx = {}
# vectors = bcolz.carray(np.zeros(1), rootdir=f'german_6B.300.dat', mode='w')

In [ ]:
# with open(f'german_glove_300d.txt', 'rb') as f:
#     for l in f:
#         line = l.decode().split()
#         word = line[0]
#         words.append(word)
#         word2idx[word] = idx
#         idx += 1
#         #Making size of embedding 48
#         vect = np.array(line[1:]).astype(np.float)
#         vectors.append(vect)

In [ ]:
# vectors = bcolz.carray(vectors[1:].reshape((-1, 300)), rootdir=f'german_6B.300.dat', mode='w')
# vectors.flush()
# pickle.dump(words, open(f'german_words.pkl', 'wb'))
# pickle.dump(word2idx, open(f'german_word2idx.pkl', 'wb'))

In [2]:
# german_vectors = bcolz.open(f'german_6B.300.dat')[:]
german_words = pickle.load(open(f'german_words.pkl', 'rb'))
# german_word2idx = pickle.load(open(f'german_word2idx.pkl', 'rb'))

# german_vectors1 = pickle.load(open(f'german_vectors1.pkl', 'rb'))
# german_vectors2 = pickle.load(open(f'german_vectors2.pkl', 'rb'))
# german_vectors3 = pickle.load(open(f'german_vectors3.pkl', 'rb'))
# german_vectors4 = pickle.load(open(f'german_vectors4.pkl', 'rb'))
# german_glove = {w: german_vectors[german_word2idx[w]] for w in german_words}

In [ ]:
# gv = german_vectors[1200000:]

In [ ]:
# german_vectors.shape, len(german_words), len(german_word2idx)

In [ ]:
# gv.shape

In [ ]:
# pickle.dump(gv, open(f'german_vectors4.pkl', 'wb'))

In [ ]:
# german_vectors1.shape, german_vectors2.shape, german_vectors3.shape, german_vectors4.shape

# English  Sentence Preprocessing

In [ ]:
english_sentence_vectors = []
with open(f'train.en', 'rb') as f:
    for l in f:
        line = l.decode()        
        english_sentence_vectors.append(line)

In [ ]:
english_sentence_vectors = english_sentence_vectors[:2000]

In [ ]:
english_matrixOfSentenceEmbeddings = []
wordembedding_matrix = torch.tensor([]).double()
unk = np.random.normal(scale=0.6, size=(300, ))


for i in english_sentence_vectors:
    temp = i.split()
    wordembedding_matrix = torch.tensor([]).double()
    for j in range(len(temp)):
        try:
            wordembedding_matrix = torch.cat([wordembedding_matrix, torch.from_numpy(english_glove[temp[j]]).reshape(1,-1)])
        except KeyError: 
            wordembedding_matrix = torch.cat([wordembedding_matrix, torch.from_numpy(unk).reshape(1,-1)])
    english_matrixOfSentenceEmbeddings.append(wordembedding_matrix)
   



In [ ]:
pickle.dump(english_matrixOfSentenceEmbeddings, open(f'english_matrixOfSentenceEmbeddings.pkl', 'wb'))
we = pickle.load(open(f'english_matrixOfSentenceEmbeddings.pkl', 'rb'))

# German Sentence Preprocessing

In [ ]:
german_sentence_vectors = []
with open(f'train.de', 'rb') as f:
    for l in f:
        line = l.decode()        
        german_sentence_vectors.append(line)

In [ ]:
german_sentence_vectors = german_sentence_vectors[:2000]

In [ ]:
german_matrixOfSentenceEmbeddings = []
wordembedding_matrix = torch.tensor([]).double()
unk = np.random.normal(scale=0.6, size=(300, ))


for i in german_sentence_vectors:
    temp = i.split()
    wordembedding_matrix = torch.tensor([]).double()
    for j in range(len(temp)):
        try:
            wordembedding_matrix = torch.cat([wordembedding_matrix, torch.from_numpy(german_glove[temp[j]]).reshape(1,-1)])
        except KeyError: 
            wordembedding_matrix = torch.cat([wordembedding_matrix, torch.from_numpy(unk).reshape(1,-1)])
    german_matrixOfSentenceEmbeddings.append(wordembedding_matrix)
    

In [ ]:
pickle.dump(german_matrixOfSentenceEmbeddings, open(f'german_matrixOfSentenceEmbeddings.pkl', 'wb'))
lione = pickle.load(open(f'german_matrixOfSentenceEmbeddings.pkl', 'rb'))